In [24]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [25]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BOG'
    #year-month-day
todays_date = '2022-08-20' 
#non_buyers_date = '2022-07-06' #1 months
customer_date_desde = '2022-06-20' #3 months
    #data for sku sales data
#sku_date_desde = '2022-06-06' #2 months

t_cambio = 3.88 #COL=3000 // MX=19.19 // BR=3.88

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [26]:
query = """
SELECT DISTINCT
    bo.submit_date,
    TO_CHAR(bo.submit_date, 'YYYY-mm') AS month,
    bo.order_id,
    bo.customer_id,
    boi.name,
    (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio} as gmv_usd,
    COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) as discount_applied,
    COALESCE(foi.total_tax_iva/{tipo_cambio},0) as taxes,
    boi.quantity* foi.step_unit as cant,
    UPPER(fpro.unit) as unidades,
    bs.sku_id,
    bs.addl_product_id as padre_sku_id,
    s.site_identifier_value as region_code,
    CASE
        WHEN COALESCE(bcat2.name, bcat.name) ILIKE 'Proteínas%' THEN 'Proteínas'
        ELSE COALESCE(bcat2.name, bcat.name) 
    END AS category,
    CASE
        WHEN bcat.name ILIKE 'Pollo%' THEN 'Pollo'
        WHEN bcat.name ILIKE 'Res%' THEN 'Res'
        WHEN bcat.name ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE bcat.name
    END AS subcat,
    baid.adjustment_reason
FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                 fow        ON fow.owner_id = ffg.owner_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_product"               fpro       ON fpro.product_id = bp.product_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_business_type"         bt         ON fc.business_type_id = bt.business_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat2      ON bcx.category_id = bcat2.category_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(bo.submit_date) between '{desde}' AND '{hasta}'
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value IN ('{ciudad}')
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   AND fc.business_type_id IN (1)
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
""".format(tipo_cambio=t_cambio, desde=customer_date_desde, hasta=todays_date, ciudad=city_code)

data_ventas = read_connection_data_warehouse.runQuery(query)
data_ventas[["gmv_usd", "cant"]] = data_ventas[["gmv_usd", "cant"]].astype(float)
data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [29]:
len(data_ventas[data_ventas.category == 'Frutas & Verduras']['customer_id'].unique())

15047

In [12]:
clients = [171043844,42075587,36173276,94273119,16030964,9748730,30346656,24267167,23307010,71437898,19592841,151325085,73463992,18968182,61672142,47577505,
           40308942,33769755,-207621,65497560,105629899,-227499,17848390,161940203,60659548,19871015,152332014,78674091,17412555,-214387,43137535,39855245,66400757,
           41475532,59317457,45264674,-214201,15327736,16187033,147366607
]

In [18]:
data_ventas.category.unique()

array(['Bebidas', 'Frutas & Verduras', 'Carne, Pollo & Pescados',
       'Lácteos & Huevos', 'Abarrotes', 'Desechables', 'Aseo e Higiene',
       'Congelados'], dtype=object)

In [13]:
data_ventas[data_ventas.category == 'Frutas & Verduras']['subcat'].unique()

array(['Verduras', 'Tubérculos', 'Frutas', 'Especies'], dtype=object)

In [14]:
subcats = ['Verduras', 'Frutas', 'Tubérculos', 'Frutas Jugo', 'Especias']
data_ventas[(data_ventas.customer_id.isin(clients)) & (data_ventas.subcat.isin(subcats))] 

,submit_date,month,order_id,customer_id,name,gmv_usd,discount_applied,taxes,cant,unidades,sku_id,padre_sku_id,region_code,category,subcat,adjustment_reason


In [15]:
a = data_ventas[(data_ventas.customer_id.isin(clients)) & (data_ventas.subcat.isin(subcats))].groupby(by=['padre_sku_id']).apply(
    lambda d: pd.Series(
        {
            "gmv_usd": d["gmv_usd"].sum(),
            "cant": d["cant"].sum(),
            "customer_id": d["customer_id"].nunique(),
            "avg_gmv": d["gmv_usd"].sum() / d["customer_id"].nunique(),
            "avg_cant": d["cant"].sum() / d["customer_id"].nunique(),
            "discount_usd": d["discount_applied"].sum(),
            "avg_discount": float(d["discount_applied"].sum() / d["customer_id"].nunique()),
            "% dct": float(d["discount_applied"].sum()) *100.0 / d["gmv_usd"].sum(),
            "skus": "#".join([str(x) for x in d["sku_id"].unique()])
            
        }
    )
).reset_index()

In [16]:
a.describe()

,padre_sku_id,gmv_usd,cant,customer_id,avg_gmv,avg_cant,avg_discount,% dct
count,130.000000,1.300000e+02,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000
mean,-113218.200000,1.606627e+06,3026.412538,192.761538,6213.233565,6.745618,164.599825,2.797038
std,133736.241765,6.550206e+06,20991.678851,329.714366,7334.453313,15.220321,219.173811,2.800206
min,-201313.000000,4.922680e+03,1.000000,1.000000,570.401513,0.722222,0.000000,0.000000
25%,-200516.000000,1.260321e+05,81.072500,30.500000,2763.268520,2.083438,13.517010,0.429624
50%,-200104.000000,3.251767e+05,254.500000,77.500000,4700.562315,3.387827,99.612980,2.342688
75%,28662.250000,1.037091e+06,1044.372500,202.250000,7054.724780,5.967100,217.711779,4.243663
max,305509.000000,7.239184e+07,238954.190000,2092.000000,66966.740283,129.406780,1171.546390,14.345833


In [19]:
inferior, superior = np.percentile(a['customer_id'],[50, 100]) #rango entre los skus a tomar

skus_offer = ''
for i in a[(a.customer_id>inferior) & (a.customer_id<superior)]['padre_sku_id'].unique():
    skus_offer = skus_offer + a[a.padre_sku_id == i]['skus'].values[0 ] + '#'
skus_offer = skus_offer[:-1]

In [20]:
skus_offer

'-302949#-303393#-302661#-302130#-300957#-300543#210159#212438#210161#62912#-300510#142264#-302016#139111#-302343#-302283#-302925#-302103#-302100#-300795#-302544#-302916#-302412#-300453#63754#-302256#160938#-301668#-301371#-300888#-301185#-301143#94846#-302562#-300207#-301077#-301137#-300927#286789#-302694#206639#-302556#112755#-300477#-300879#62906#-300753#-300777#62904#-300213#-300222#-300225#-300231#-300249#-302943#-302901#-302898#-302946#327559#327561#327556#-300387#-302028#-302025#-302595#112739#134243#-302484#-301326#-302034#196351#-302919#192764#63744#134238#-302220#63739#-300783#-302208#101969#-303369#-302466#206651#-301269#62896#-302703#-300645#-300657#16211#16209#16224#18439#18446#26107#287738#36878#42875#47269#55848#63788#258690#73211#258696#73267#92108#97944#131193#142749#142755#214690#214694#214692#216258#216264#216262#258799#287277#401340#563293#401348#563300#401346#563298'

In [60]:
# dict_clients = {'clientes': list(clientes_offer)}
# df_clients = pd.DataFrame(dict_clients)
# dict_control = {'control': list(control)}
# df_control = pd.DataFrame(dict_control)
# dict_test = {'test': list(test)}
# df_test = pd.DataFrame(dict_test)

In [21]:
global_segment_list = [{'name':'20220819_BAQ_experiment_churn_graduados', 'customersIds': list([int(x) for x in clients])}]

In [22]:
csv_file = {} 
csv_file['offer_name'] = '20220819_BAQ_experiment_churn_graduados'
csv_file['offer_description'] = None
csv_file['discount'] = 11
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = '20-08-2022'
csv_file['end_date'] = '28-08-2022'
csv_file['max_uses_per_order'] = 100
csv_file['max_uses_per_customer'] = 1
csv_file['customer_segment_id'] = None
csv_file['sku_id'] = skus_offer
global_offer_csv = pd.DataFrame(csv_file, index=[0])


In [23]:
# #//--------------------------------------------
# #//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_experiment_churn_graduados.csv', index = False)

#df_clients.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_clientes_offer_J1.csv', index = False)
#df_control.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_control_J1.csv', index = False)
#df_test.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_test_J1.csv', index = False)


with open(f'SD/export_data/Below_Media/nb_{city_code}_experiment_churn_graduados.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'SD/export_data/Below_Media/nb_{city_code}_experiment_churn_graduados.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'SD/Done {city_code}')

SD/Done BAQ
